In [33]:
import os
import sys
import datetime
from pathlib import Path
from typing import List, Tuple, Sequence, Mapping, Optional, Dict, Union
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import BoundaryNorm
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import statsmodels.api as sm
import geopandas as gpd
from libpysal import weights
from libpysal.weights import Queen, Rook
import spreg
from spreg import OLS as PYSAL_OLS
from spreg.skater_reg import Skater_reg
from mgwr.sel_bw import Sel_BW
from mgwr.gwr import MGWR
sys.path.insert(
    0,
    r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Guo_et_al"
)
import Algorithm9
from Algorithm9 import kmodels, split_components, greedy_merge
import Network9
from Network9 import Test_Equations, regression_error
import GridData9
from GridData9 import *
np.random.seed(0)
import statsmodels.api as sm

In [3]:
def plot_result(
    dirs: Sequence[str],
    file_names: Sequence[str],
    fields: Sequence[str],
    *,
    ncols = 2,
    colorbar = False,
    cmap = "viridis",
    fig_size = (3, 3),
):

    entries: list[tuple[np.ndarray, str]] = []  # (grid, title)

    for d in dirs:
        d_base = os.path.basename(d.rstrip("/\\"))
        for fname in file_names:
            path = os.path.join(d, fname)
            df = pd.read_csv(path, sep="\t")
            
            side_x = int(df["u"].max()) + 1
            side_y = int(df["v"].max()) + 1

            df_sorted = df.sort_values(["v", "u"])
            file_stem = os.path.splitext(fname)[0]
            base_title = f"{d_base} | {file_stem}"

            for field in fields:
                arr = df_sorted[field].to_numpy()
                grid = arr.reshape((side_y, side_x))
                entries.append((grid, f"{base_title}\n{field}"))

    if not entries:
        raise ValueError("No valid (dir, file, field) entries to plot.")

    nplots = len(entries)
    nrows = int(np.ceil(nplots / ncols))
    fig, axes = plt.subplots(
        nrows, ncols,
        figsize=(fig_size[0] * ncols, fig_size[1] * nrows),
        #figsize=(12,12),
        squeeze=False
    )

    for i, (grid, title) in enumerate(entries):
        ax = axes[i // ncols, i % ncols]
        im = ax.imshow(grid, origin="lower", cmap=cmap)
        ax.set_title(title)
        ax.set_xticks([]); ax.set_yticks([])
        if colorbar:
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    for j in range(i + 1, nrows * ncols):
        fig.delaxes(axes[j // ncols, j % ncols])

    plt.tight_layout()
    plt.show()

### Guo et al (Kmodels)

Test for 1 feat continuous GFS test

In [5]:
def Pos_Encode(r,c,side):
    return r*side+c

def Pos_Decode(code,side):
    return code//side, code%side

def run_kmodels(data_dir, file_names,
                n_regions=5, micro_clusters=None, min_region=20,
                save_region_label=False, verbose=True,
                output_dir=None):

    if micro_clusters is None:
        micro_clusters = 2 * n_regions

    results = {}
    for fname in file_names:
            df = pd.read_csv(os.path.join(data_dir, fname), sep="\t")

            X = df[["feat1", "feat2"]].to_numpy()
            y = df["y"].to_numpy()
            # Xs = preprocessing.StandardScaler().fit_transform(X)
            # ys = preprocessing.StandardScaler().fit_transform(y.reshape(-1, 1)).ravel()
            Xarr = np.hstack([np.ones((len(y), 1)), X])
            Yarr = y

            side_x = int(df["u"].max()) + 1
            side_y = int(df["v"].max()) + 1
            
            w = weights.lat2W(side_y, side_x).symmetrize()

            # --- KModels → split → merge ---
            clabel, _ = kmodels(Xarr, Yarr, micro_clusters, w, init_stoc_step=False, verbose=False)
            slabel    = split_components(w, clabel)
            rlabel, rcoeff, _ = greedy_merge(Xarr, Yarr, n_regions, w, slabel,
                                             min_size=min_region, verbose=False)

            coefs = np.vstack([rcoeff[lab] for lab in rlabel])
            df_out = df.copy()
            df_out["a1_result"] = coefs[:, 1]
            df_out["a2_result"] = coefs[:, 2]
            df_out["b_result"]  = coefs[:, 0]
            if save_region_label:
                df_out["region_label"] = rlabel

            out_name = fname.replace(".txt", "_KM_result.txt")
            out_path = os.path.join(out_dir, out_name)
            df_out.to_csv(out_path, sep="\t", index=False)
            if verbose:
                print(f"[OK] {out_path}")

            results[fname] = df_out
    return results, rcoeff, rlabel, coefs

In [26]:
#df = r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Data_gen"
data_dir   = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_gen"
file_names = [
    "df_con_uni.txt",
    "df_con_grf.txt",
    "df_discon_uni.txt",
    "df_discon_grf.txt",
    "df_multi_uni.txt",
    "df_multi_grf.txt",
    "df_vor_grf.txt",
    "df_vor_uni.txt"
]
out_dir = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM"
res = run_kmodels(n_regions=10, data_dir = data_dir,file_names = file_names, output_dir = out_dir,save_region_label=True)

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM\df_con_uni_KM_result.txt
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM\df_con_grf_KM_result.txt
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM\df_discon_uni_KM_result.txt
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM\df_discon_grf_KM_result.txt
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM\df_multi_uni_KM_result.txt
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM\df_multi_grf_KM_result.txt
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM\df_vor_grf_KM_result.txt
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/KM\df_vor_uni_KM_result.txt


### Skater Regression

In [103]:
def run_skreg(
    data_dir: str,
    file_names: Sequence[str] | str,
    n_regions = 5,
    quorum = 20,
    save_region_label = False,
    output_dir = None,
    verbose = False,
):
    if isinstance(file_names, str):
        files = [file_names]
    else:
        files = list(file_names)

    out_dir = os.path.join(data_dir, output_dir) if output_dir else data_dir
    os.makedirs(out_dir, exist_ok=True)

    results: dict[str, pd.DataFrame] = {}

    for fname in files:
        path = os.path.join(data_dir, fname)
        df = pd.read_csv(path, sep="\t")
        #feat_cols = [c for c in df if c.startswith("feat")]
        X = df[["feat1", "feat2"]].to_numpy()
        y = df["y"].to_numpy()

        side_x = int(df["u"].max()) + 1
        side_y = int(df["v"].max()) + 1
        w = weights.lat2W(side_y, side_x).symmetrize()
        X_std = StandardScaler().fit_transform(X)
        #y_std = StandardScaler().fit_transform(y.reshape(-1, 1)).ravel()
        
        skater = Skater_reg()
        res = skater.fit(
            n_regions,
            w,
            X_std,                                
            {"reg": PYSAL_OLS, "y": y, "x": X},
            quorum=quorum,
            verbose=False
        )
        labels = res._trace[-1][0].astype(int)
        # each sklearn regiems regression
        region_coefs: dict[int, np.ndarray] = {}
        for r in np.unique(labels):
            idx = (labels == r)
            Xr, yr = X[idx, :], y[idx]
            lr = LinearRegression(fit_intercept=True).fit(Xr, yr)
            region_coefs[r] = np.array([lr.intercept_, *lr.coef_])  # [b, a1, a2]

        coefs = np.vstack([region_coefs[int(lab)] for lab in labels])  # (N,3)
        df_out = df.copy()
        df_out["b_result"]  = coefs[:, 0]
        df_out["a1_result"] = coefs[:, 1]
        df_out["a2_result"] = coefs[:, 2]
        if save_region_label:
            df_out["region_label"] = labels

        out_name = fname.replace(".txt", "_SKREG_result.txt")
        out_path = os.path.join(out_dir, out_name)
        df_out.to_csv(out_path, sep="\t", index=False)
        if verbose:
            print(f"[OK] {out_path}")
        results[fname] = df_out

    return results

In [203]:
data_dir = r".../Data_gen"
out_dir = r".../Data_result/SKREG"
files = [
    "df_con_uni.txt",
    "df_discon_uni.txt",
    "df_multi_uni.txt",
    "df_vor_uni.txt"
]

res = run_skreg(
    data_dir=data_dir,
    file_names=files,              
    n_regions=15,
    quorum=20,
    save_region_label=False,
    output_dir=out_dir,
    verbose=True
)

KeyboardInterrupt: 

### MGWR

In [4]:
ArrayLike = Union[np.ndarray, List[float], float]

def run_mgwr(
    data_dir: str,
    file_names: Sequence[str] | str,
    *,
    feature_cols: Sequence[str],                 
    y_col: str = "y",
    coord_cols: Tuple[str, str] = ("x_coord", "y_coord"),
    output_dir: Optional[str] = None,            
    multi: bool = True,                          
    fixed: bool = False,                         
    kernel: str = "bisquare",                   
    spherical: bool = False,                     
    bws: Optional[ArrayLike] = None,             
) -> Dict[str, pd.DataFrame]:

    files = [file_names] if isinstance(file_names, str) else list(file_names)
    out_dir = data_dir if output_dir is None else output_dir
    os.makedirs(out_dir, exist_ok=True)

    results: Dict[str, pd.DataFrame] = {}

    for fname in files:
        path = os.path.join(data_dir, fname)
        if not os.path.exists(path):
            raise FileNotFoundError(path)
        df = pd.read_csv(path, sep="\t")

        coords = df.loc[:, coord_cols].to_numpy()
        y = df.loc[:, y_col].to_numpy().reshape(-1, 1)
        X_feat = df.loc[:, feature_cols].to_numpy()
        n = len(df)
        X = np.hstack([np.ones((n, 1)), X_feat])

        if bws is None:
            bw_selector = Sel_BW(
                coords, y, X,
                multi=multi, fixed=fixed,
                kernel=kernel, spherical=spherical,
                constant=False
            )
            _bws = bw_selector.search()
        else:
            _bws = bws

        model = MGWR(
            coords, y, X, bw_selector,
            fixed=fixed, kernel=kernel, spherical=spherical,
            constant=False
        )
        res = model.fit()
        params = res.params  # (n, p+1): [b, a1, a2, ...]

        df_out = df.copy()
        df_out["b_result"]  = params[:, 0]
        df_out["a1_result"] = params[:, 1]  
        df_out["a2_result"] = params[:, 2]  

        stem = os.path.splitext(fname)[0]
        out_path = os.path.join(out_dir, f"{stem}_MGWR_result.txt")
        df_out.to_csv(out_path, sep="\t", index=False)
        results[fname] = df_out
        print(f"[OK] {out_path}")

    return results


In [6]:
data_dir = r".../Data_gen"
out_dir = r".../Data_result/MGWR"
files = [
    "df_con_uni.txt",
    "df_discon_uni.txt",
    "df_multi_uni.txt",
    "df_vor_uni.txt"
]

In [7]:
run_mgwr(
    data_dir=data_dir,
    file_names=files,
    feature_cols=["feat1", "feat2"],
    y_col="y",
    coord_cols=("spatial_x", "spatial_y"),
    output_dir=os.path.join(data_dir, "..", "Data_result", "MGWR"),
    multi=True,
    fixed=False,
    kernel="bisquare",
    spherical=False,
)

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_gen\..\Data_result\MGWR\df_con_uni_MGWR_result.txt


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

...

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_gen\..\Data_result\MGWR\df_multi_uni_MGWR_result.txt


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_gen\..\Data_result\MGWR\df_vor_uni_MGWR_result.txt


{'df_con_uni.txt':        u   v  spatial_x  spatial_y   a1_list   a2_list  b_list     feat1  \
 0      0   0   0.000000        0.0 -0.886420 -0.625670       3  1.643736   
 1      1   0   0.204082        0.0 -0.880833 -0.645478       3 -0.366729   
 2      2   0   0.408163        0.0 -0.862968 -0.665857       3  2.151588   
 3      3   0   0.612245        0.0 -0.834523 -0.686958       3  1.184208   
 4      4   0   0.816327        0.0 -0.797362 -0.708920       3 -2.434936   
 ...   ..  ..        ...        ...       ...       ...     ...       ...   
 2495  45  49   9.183673       10.0  1.457911 -0.333272       3  0.132849   
 2496  46  49   9.387755       10.0  1.460655 -0.351329       3  0.313519   
 2497  47  49   9.591837       10.0  1.441659 -0.363042       3 -1.729553   
 2498  48  49   9.795918       10.0  1.400341 -0.368267       3  1.916705   
 2499  49  49  10.000000       10.0  1.336925 -0.366932       3 -0.008400   
 
          feat2         y  b_result  a1_result  a2_resul

# Index

## KM

In [32]:
def get_bic_r2 (Xarr, Yarr, labels, rcoeff, per_region_sigma=True):
    
    N, p = Xarr.shape
    lab = np.asarray(labels)

    beta_mat = np.vstack([rcoeff[int(lb)] for lb in lab])  # (N,p) [b,a1,a2,...]
    yhat = np.sum(Xarr * beta_mat, axis=1)
    resid = Yarr - yhat

    SSE = float(np.sum(resid ** 2))
    SST = np.sum((Yarr - Yarr.mean())**2)

    R = len(np.unique(lab))
    k = R * p

    BIC = N * np.log(SSE / N) + k * np.log(N)
    R_2 = 1 - SSE/SST

    return BIC, R_2, k, SSE/2500


def run_kmodels_bic(
    data_dir, file_names,
    n_regions=5, micro_clusters=None, min_region=20,
    save_region_label=False, verbose=True, output_dir=None,
    R_min: int = 2, R_max: int = 15, per_region_sigma: bool = True
):

    R_min = int(R_min); R_max = int(R_max)
    if R_min < 1: R_min = 1
    if R_max < R_min: R_min, R_max = R_max, R_min

    files = [file_names] if isinstance(file_names, str) else list(file_names)
    out = {}

    for fname in files:
        df = pd.read_csv(os.path.join(data_dir, fname), sep="\t")

        X = df[["feat1", "feat2"]].to_numpy()
        y = df["y"].to_numpy()
        # Xs = preprocessing.StandardScaler().fit_transform(X)
        # ys = preprocessing.StandardScaler().fit_transform(y.reshape(-1, 1)).ravel()
        Xarr = np.hstack([np.ones((len(y), 1)), X])
        Yarr = y
        side_x = int(df["u"].max()) + 1
        side_y = int(df["v"].max()) + 1
        assert side_x * side_y == len(df),
        w = weights.lat2W(side_y, side_x).symmetrize()

        rows = []
        for R in range(R_min, R_max + 1):
            try:
                K_micro = 2 * R
                clabel, _ = kmodels(Xarr, Yarr, K_micro, w,
                                    init_stoc_step=False, verbose=False)
                slabel = split_components(w, clabel)

                rlabel, rcoeff, _ = greedy_merge(
                    Xarr, Yarr, R, w, slabel, min_size=min_region, verbose=False
                )
                bic, r_2, k, MSE = get_bic_r2(Xarr, Yarr, rlabel, rcoeff)
                
                rows.append({"n_regions": R, "BIC": bic, "R2": r_2, "MSE": MSE})
                
                if verbose:
                    print(f"{fname} | n_regions={R:2d} (K_micro={K_micro:2d})  BIC={bic:.6f} R2={r_2:.6f} MSE={MSE:.6f} ")
            except Exception as e:
                rows.append({"n_regions": R, "BIC": np.nan})
                if verbose:
                    print(f"{fname} | n_regions={R:2d} (K_micro={2*R:2d})  FAILED → {e}")

        out[fname] = pd.DataFrame(rows)

    return out


In [31]:
data_dir   = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_gen"
files = [
    "df_con_uni.txt",
    "df_discon_uni.txt",
    "df_multi_uni.txt",
    "df_vor_uni.txt"
]


bic_km_2 = run_kmodels_bic(
    data_dir, files,
    min_region=20,
    R_min=2, R_max=15,
    verbose=True
)


df_con_uni.txt | n_regions= 2 (K_micro= 4)  BIC=4087.653847 R2=0.428319 MSE=5.034347 
df_con_uni.txt | n_regions= 3 (K_micro= 6)  BIC=2919.723785 R2=0.645035 MSE=3.125902 
df_con_uni.txt | n_regions= 4 (K_micro= 8)  BIC=2469.867122 R2=0.706262 MSE=2.586722 
df_con_uni.txt | n_regions= 5 (K_micro=10)  BIC=2174.554217 R2=0.741428 MSE=2.277042 
df_con_uni.txt | n_regions= 6 (K_micro=12)  BIC=1885.145209 R2=0.771846 MSE=2.009177 
df_con_uni.txt | n_regions= 7 (K_micro=14)  BIC=1552.410804 R2=0.802144 MSE=1.742363 
df_con_uni.txt | n_regions= 8 (K_micro=16)  BIC=1578.210125 R2=0.801960 MSE=1.743986 
df_con_uni.txt | n_regions= 9 (K_micro=18)  BIC=1442.540713 R2=0.814173 MSE=1.636429 
df_con_uni.txt | n_regions=10 (K_micro=20)  BIC=1206.069237 R2=0.832525 MSE=1.474825 
df_con_uni.txt | n_regions=11 (K_micro=22)  BIC=1306.122473 R2=0.827315 MSE=1.520701 
df_con_uni.txt | n_regions=12 (K_micro=24)  BIC=1158.150577 R2=0.838760 MSE=1.419911 
df_con_uni.txt | n_regions=13 (K_micro=26)  BIC=1113.7

In [95]:
def export_bic(
    bic_dict: dict,
    out_path: str | None = None,
    split_dir: str | None = None,
    *,
    dropna: bool = False,
    ensure_int_regions: bool = True,
    drop_file_col_when_split: bool = True
) -> pd.DataFrame:

    rows = []
    for fname, df in bic_dict.items():
        df2 = df.copy()
        df2.insert(0, "file", fname)
        rows.append(df2)

    big = pd.concat(rows, ignore_index=True)
    required = {"file", "n_regions", "BIC"}
    if ensure_int_regions:
        big["n_regions"] = big["n_regions"].astype("Int64").astype("float").astype(int)
    if dropna:
        big = big.dropna(subset=["BIC"])
    big = big.sort_values(["file", "n_regions"], kind="mergesort")
    if out_path:
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        big.to_csv(out_path, sep="\t", index=False)
    if split_dir:
        os.makedirs(split_dir, exist_ok=True)
        for f, sub in big.groupby("file", sort=True):
            sub_out = sub if not drop_file_col_when_split else sub[["n_regions", "BIC", "R2", "MSE"]]
            out_file = os.path.join(split_dir, f)
            os.makedirs(os.path.dirname(out_file), exist_ok=True)
            sub_out.to_csv(out_file, sep="\t", index=False)
            print(f"[OK]：{out_file}")
    return big

## SKEG

In [46]:
def get_bic_r2_sk (labels, X_feat, y):

    labels = np.asarray(labels)
    uniq = np.unique(labels)
    X = np.asarray(X_feat, float)
    y = np.asarray(y, float).ravel()

    N, p0 = X_feat.shape
    P = p0 + 1

    SSE = 0.0
    for r in uniq:
        idx = (labels == r)
        Xr = X_feat[idx, :]
        yr = y[idx]
        Xrd = np.c_[np.ones(len(yr)), Xr]
        beta, *_ = np.linalg.lstsq(Xrd, yr, rcond=None)
        SSE += float(np.sum((yr - Xrd @ beta) ** 2))

    y_center = y - y.mean()
    SST = np.sum((y - y.mean())**2)
    r2 = 1.0 - SSE / SST

    R = len(uniq)
    k = R * P
    bic = N * np.log(SSE / N) + k * np.log(N)
    return bic, r2, SSE/N

In [44]:
def sweep_skater_global_bic_singlefit(
    data_dir: str,
    file_names,
    *,
    R_min: int = 2,
    R_max: int = 15,
    quorum: int = 20,
    verbose: bool = True,
):
    files = [file_names] if isinstance(file_names, str) else list(file_names)
    out = {}

    for fname in files:
        path = os.path.join(data_dir, fname)
        df = pd.read_csv(path, sep="\t")
        # X, y
        X = df[["feat1", "feat2"]].to_numpy()
        y = df["y"].to_numpy()
        side_x = int(df["u"].max()) + 1
        side_y = int(df["v"].max()) + 1
        w = weights.lat2W(side_y, side_x).symmetrize()
        X_std = StandardScaler().fit_transform(X)
        rows = []
        try:
            skater = Skater_reg()
            res = skater.fit(
                R_max, w, X_std,
                {"reg": spreg.OLS, "y": y, "x": X},
                quorum=quorum
            )
            keep = {}
            for rec in res._trace:
                labels = np.asarray(rec[0]).astype(int)
                K = np.unique(labels).size
                if R_min <= K <= R_max:
                    keep[K] = labels
            for K in sorted(keep):
                bic, r2, MSE = get_bic_r2_sk(keep[K], X, y)
                rows.append({"n_regions": K, "BIC": bic, "R2": r2, "MSE": MSE})
                if verbose:
                    print(f"{fname} | K={K:2d}  BIC={bic:.6f}  R2={r2:.6f}  MSE={MSE:.6f}")
        except Exception as e:
            if verbose:
                print(f"{fname} | single fit FAILED → {e}")
        out[fname] = pd.DataFrame(rows).sort_values("n_regions").reset_index(drop=True)
    return out

In [45]:
data_dir   = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_gen"
files = [
    "df_con_uni.txt",
    #"df_con_grf.txt",
    "df_discon_uni.txt",
    #"df_discon_grf.txt",
    "df_multi_uni.txt",
    #"df_multi_grf.txt",
    #"df_vor_grf.txt",
    "df_vor_uni.txt"
]


bic_sk = sweep_skater_global_bic_singlefit(
    data_dir, files,
    R_min=2, R_max=15,
    quorum=20,
    verbose=True
)

df_con_uni.txt | K= 2  BIC=4601.167752  R2=0.297963  MSE=6.182294
df_con_uni.txt | K= 3  BIC=3780.140258  R2=0.499210  MSE=4.410065
df_con_uni.txt | K= 4  BIC=2888.658027  R2=0.652694  MSE=3.058449
df_con_uni.txt | K= 5  BIC=2703.448766  R2=0.680508  MSE=2.813517
df_con_uni.txt | K= 6  BIC=2509.216714  R2=0.707153  MSE=2.578876
df_con_uni.txt | K= 7  BIC=2334.028685  R2=0.729523  MSE=2.381879
df_con_uni.txt | K= 8  BIC=2235.056594  R2=0.742451  MSE=2.268030
df_con_uni.txt | K= 9  BIC=2130.140492  R2=0.755344  MSE=2.154495
df_con_uni.txt | K=10  BIC=2059.162106  R2=0.764415  MSE=2.074616
df_con_uni.txt | K=11  BIC=1998.038543  R2=0.772253  MSE=2.005589
df_con_uni.txt | K=12  BIC=1941.140613  R2=0.779458  MSE=1.942139
df_con_uni.txt | K=13  BIC=1898.800280  R2=0.785188  MSE=1.891679
df_con_uni.txt | K=14  BIC=1862.116301  R2=0.790295  MSE=1.846704
df_con_uni.txt | K=15  BIC=1828.503777  R2=0.795030  MSE=1.805015
df_discon_uni.txt | K= 2  BIC=7742.371081  R2=0.180737  MSE=21.718637
df_dis

## MGWR

In [9]:
def run_mgwr_bic(
    data_dir: str,
    file_names: Sequence[str] | str,
    *,
    feature_cols: Sequence[str],                 
    y_col: str = "y",
    coord_cols: Tuple[str, str] = ("spatial_x", "spatial_x"),
    output_dir: [str] = None,            
    multi: bool = True,                          
    fixed: bool = False,                         
    kernel: str = "bisquare",                    
    spherical: bool = False,                     
    verbose: bool = True,
) -> Tuple[[str, pd.DataFrame], pd.DataFrame]:

    files = [file_names] if isinstance(file_names, str) else list(file_names)
    out_dir = data_dir if output_dir is None else output_dir
    os.makedirs(out_dir, exist_ok=True)

    results: Dict[str, pd.DataFrame] = {}
    metrics_rows: List[dict] = []

    for fname in files:
        path = os.path.join(data_dir, fname)
        if not os.path.exists(path):
            raise FileNotFoundError(path)
        df = pd.read_csv(path, sep="\t")

        coords  = df.loc[:, coord_cols].to_numpy()
        y       = df.loc[:, y_col].to_numpy().reshape(-1, 1)
        X  = df.loc[:, feature_cols].to_numpy()
        n       = len(df)

        # X = StandardScaler().fit_transform(X)
        # y = StandardScaler().fit_transform(y.reshape(-1, 1))
        
        # constant=False
        X = np.hstack([np.ones((n, 1)), X])

        bw_selector = Sel_BW(
            coords, y, X,
            multi=multi,
            constant=False
        )

        #kernel='bw_selector.search(criterion='BIC')
        #selector.search()

        bw_selector.search()

        model = MGWR(
            coords, y, X, bw_selector,
            constant=False
        )
        res = model.fit()

        params = res.params  
        df_out = df.copy()
        df_out["b_result"]  = params[:, 0]
        df_out["a1_result"] = params[:, 1]
        df_out["a2_result"] = params[:, 2]

        y_true = y.ravel()
        yhat   = np.asarray(res.predy).ravel()
        N      = y_true.size
        SSE    = float(np.sum((y_true - yhat)**2))
        SST    = float(np.sum((y_true - y_true.mean())**2))
        R2     = np.nan if SST == 0.0 else (1.0 - SSE / SST)

        BIC_enp = N * np.log(SSE / N) + enp * np.log(N)

        AIC_mgwr  = float(res.aic)
        AICc_mgwr = float(res.aicc)
        BIC_mgwr  = float(res.bic)

        stem = os.path.splitext(fname)[0]
        out_coef = os.path.join(out_dir, f"{stem}_MGWR_result.txt")
        df_out.to_csv(out_coef, sep="\t", index=False)

        out_metrics = os.path.join(out_dir, f"{stem}_MGWR_metrics.txt")
        pd.DataFrame([{
            "file": fname,
            "N": N,
            "ENP": enp,
            "SSE": SSE,
            "R2": R2,
            "BIC_enp": BIC_enp,
            "BIC_mgwr": BIC_mgwr,
            # "AIC": AIC_mgwr,
            # "AICc": AICc_mgwr
        }]).to_csv(out_metrics, sep="\t", index=False)

        results[fname] = df_out
        metrics_rows.append({
            "file": fname,
            "N": N,
            "ENP": enp,
            "SSE": SSE,
            "R2": R2,
            "BIC_enp": BIC_enp,
            "BIC_mgwr": BIC_mgwr,
            # "AIC": AIC_mgwr,
            # "AICc": AICc_mgwr
        })
        if verbose:
            print(f"[OK] {out_coef} | BIC_enp={BIC_enp:.6f}, R2={R2:.6f}, ENP={enp:.2f}")

    return results, metrics_rows

In [10]:
data_dir   = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_gen"
files = [
    "df_con_uni.txt",
    "df_discon_uni.txt",
    "df_multi_uni.txt",
    "df_vor_uni.txt"
]

coef_dict, metrics_df = run_mgwr_bic(
    data_dir=data_dir,
    file_names=files,
    feature_cols=["feat1","feat2"],
    y_col="y",
    coord_cols=("spatial_x","spatial_y"),
    output_dir=r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/MGWR/uni",
    fixed=False, kernel="bisquare", spherical=False,
    verbose=True
)


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/MGWR/uni\df_con_uni_MGWR_result.txt | BIC_enp=1194.905771, R2=0.865970, ENP=99.76


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

...

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/MGWR/uni\df_multi_uni_MGWR_result.txt | BIC_enp=4266.129714, R2=0.841107, ENP=190.92


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/MGWR/uni\df_vor_uni_MGWR_result.txt | BIC_enp=6524.289415, R2=0.848666, ENP=292.43


In [213]:
data_dir   = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_gen"
files = [
    "df_con_uni.txt",
    "df_con_grf.txt",
    "df_discon_uni.txt",
    "df_discon_grf.txt",
    "df_multi_uni.txt",
    "df_multi_grf.txt",
    "df_vor_grf.txt",
    "df_vor_uni.txt"
]

coef_dict, metrics_df = run_mgwr_bic(
    data_dir=data_dir,
    file_names=files,
    feature_cols=["feat1","feat2"],
    y_col="y",
    coord_cols=("spatial_x","spatial_y"),
    output_dir=r".../Data_result/MGWR",
    multi=True, fixed=False, kernel="bisquare", spherical=False,
    verbose=True
)



Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/MGWR\df_con_uni_MGWR_result.txt | BIC_enp=1194.905771, R2=0.865970, ENP=99.76


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

...

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/MGWR\df_vor_grf_MGWR_result.txt | BIC_enp=3439.147994, R2=0.819714, ENP=227.78


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/MGWR\df_vor_uni_MGWR_result.txt | BIC_enp=6524.289415, R2=0.848666, ENP=292.43


In [214]:
pd.DataFrame(metrics_df).to_csv(r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/MGWR/_MGWR_metrics_all.txt", sep="\t", index=False)

In [100]:
def best_n_regions_from_bic(bic_path: str) -> int:
    df = pd.read_csv(bic_path, sep="\t")
    row = df.sort_values(["MSE", "n_regions"], kind="mergesort").iloc[0]
    return int(row["n_regions"])

def run_km_with_best_bic(
    data_dir: str,
    files: list[str],
    bic_dir: str,
    bic_suffix: str = "_km.txt",
    out_dir: str = r"...\Data_result\sim_best",
    min_region: int = 20,
    save_region_label: bool = False,
    verbose: bool = True,
):
    os.makedirs(out_dir, exist_ok=True)
    for fname in files:
        stem = os.path.splitext(fname)[0]
        bic_path = os.path.join(bic_dir, f"{stem}{bic_suffix}")
        R = best_n_regions_from_bic(bic_path)
        if verbose:
            print(f"[KM] {fname} → best n_regions={R}  (BIC from: {bic_path})")
        results, rcoeff, rlabel, coefs = run_kmodels(
            data_dir=data_dir,
            file_names=[fname],
            n_regions=R,
            micro_clusters=2 * R,
            min_region=min_region,
            save_region_label=save_region_label,
            verbose=verbose,
            output_dir=out_dir,
        )
        df = pd.read_csv(os.path.join(data_dir, fname), sep="\t")

        X = df[["feat1", "feat2"]].to_numpy()
        y = df["y"].to_numpy()
        Xarr = np.hstack([np.ones((len(y), 1)), X])
        Yarr = y
        
        bic, r_2, k, MSE = get_bic_r2(Xarr, Yarr, rlabel, rcoeff)
                                
        if verbose:
            print(f"{fname} | n_regions={R:2d} ( BIC={bic:.6f} R2={r_2:.6f} SSE={MSE:.6f} ")

def run_skater_with_best_bic(
    data_dir: str,
    files: list[str],
    bic_dir: str,
    bic_suffix: str = "_sk.txt",
    out_dir: str = r"...\Data_result\sim_best",
    quorum: int = 20,
    save_region_label: bool = False,
    verbose: bool = True,
):
    os.makedirs(out_dir, exist_ok=True)
    for fname in files:
        stem = os.path.splitext(fname)[0]
        bic_path = os.path.join(bic_dir, f"{stem}{bic_suffix}")
        R = best_n_regions_from_bic(bic_path)
        if verbose:
            print(f"[SK] {fname} → best n_regions={R}  (BIC from: {bic_path})")

        run_skreg(
            data_dir=data_dir,
            file_names=[fname],
            n_regions=R,
            quorum=quorum,
            save_region_label=save_region_label,
            output_dir=out_dir,
            verbose=verbose,
        )

In [105]:
data_dir = r".../Data_gen"
files = [
    "df_con_uni.txt",
    #"df_con_grf.txt",
    "df_discon_uni.txt",
    #"df_discon_grf.txt",
    "df_multi_uni.txt",
    #"df_multi_grf.txt",
    #"df_vor_grf.txt",
    "df_vor_uni.txt"
]

bic_dir_km = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/km"
bic_dir_sk = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/sk"

out_dir = r"D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/sim_best/uni"

run_km_with_best_bic(
    data_dir=data_dir,
    files=files,
    bic_dir=bic_dir_km,
    bic_suffix="_km.txt",
    out_dir=out_dir,
    min_region=20,
    save_region_label=False,
    verbose=True,
)

[KM] df_con_uni.txt → best n_regions=10  (BIC from: D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/km\df_con_uni_km.txt)
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/sim_best/uni\df_con_uni_KM_result.txt
df_con_uni.txt | n_regions=10 ( BIC=1270.775105 R2=0.828133 SSE=1.513495 
[KM] df_discon_uni.txt → best n_regions=9  (BIC from: D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/km\df_discon_uni_km.txt)
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/sim_best/uni\df_discon_uni_KM_result.txt
df_discon_uni.txt | n_regions= 9 ( BIC=4044.993263 R2=0.825184 SSE=4.634358 
[KM] df_multi_uni.txt → best n_regions=8  (BIC from: D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/km\df_multi_uni_km.txt)
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/sim_best/uni\df_multi_uni_KM_result.txt
df_multi_uni.txt | n_regions= 8 ( BIC=3558.679676 R2=0.798123 SSE=3.851108 
[

In [113]:
run_skater_with_best_bic(
    data_dir=data_dir,
    files=files,
    bic_dir=bic_dir_sk,
    bic_suffix="_skreg.txt",
    out_dir=out_dir,
    quorum=20,
    save_region_label=False,
    verbose=True,
)

[SK] df_con_uni.txt → best n_regions=7  (BIC from: D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/sk\df_con_uni_skreg.txt)
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/sim_best/uni\df_con_uni_SKREG_result.txt
[SK] df_discon_uni.txt → best n_regions=10  (BIC from: D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/sk\df_discon_uni_skreg.txt)
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/sim_best/uni\df_discon_uni_SKREG_result.txt
[SK] df_multi_uni.txt → best n_regions=8  (BIC from: D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/sk\df_multi_uni_skreg.txt)
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_result/sim_best/uni\df_multi_uni_SKREG_result.txt
[SK] df_vor_uni.txt → best n_regions=6  (BIC from: D:/wanghanbin/Linear Regression Tree/code/comparision_paper/BIC/sk\df_vor_uni_skreg.txt)
[OK] D:/wanghanbin/Linear Regression Tree/code/comparision_paper/Data_

# Empirical

# Vote

In [28]:
import geopandas as gpd
from libpysal.weights import Queen, Rook

vote_path = r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Data\vote_std.txt"
df_vote = pd.read_csv(vote_path, sep="\t").sort_values("county_id").reset_index(drop=True)

us_county = gpd.read_file("https://raw.github.com/Ziqi-Li/gis5122/master/data/us_counties.geojson")
us_county = us_county.sort_values("county_id").reset_index(drop=True)

poly = us_county.merge(df_vote[["county_id"]], on="county_id", how="inner")

w_q = Rook.from_dataframe(poly)

DataSourceError: HTTP response code: 301

In [ ]:
mask = ~pd.Series(range(len(poly))).isin(islands)
poly = poly.loc[mask].reset_index(drop=True)
df_vote = df_vote.loc[mask].reset_index(drop=True)

w_q = Rook.from_dataframe(poly, use_index=False) 

In [51]:
df_vote.to_csv(r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Data\vote_std_rook.txt",sep="\t")

## KM

In [40]:
def run_kmodels_empirical(
    df,
    *,
    y_col: str = "new_pct_dem",
    w,
    feature_cols: Sequence[str] = (
        "proj_X","proj_Y","pct_black","pct_hisp","pct_bach",
        "median_income","ln_pop_den","pct_3rd_party","pct_fb"
    ),
    coord_cols: Tuple[str, str] = ("proj_X","proj_Y"),
    n_regions: int = 20,
    min_region: int = 50,
    micro_clusters: Optional[int] = None,
    k_neighbors: int = 8,
    standardize: bool = True,
    save_region_label: bool = True,
    out_path: Optional[str] = None
) -> pd.DataFrame:
    n = len(df)

    X_feat = df.loc[:, feature_cols].to_numpy(dtype=float)
    y_raw  = df.loc[:, y_col].to_numpy(dtype=float).ravel()
    coords = df.loc[:, coord_cols].to_numpy(dtype=float)

    if standardize:
        sx = StandardScaler().fit(X_feat)
        sy = StandardScaler().fit(y_raw.reshape(-1,1))
        X_std = sx.transform(X_feat)
        y_std = sy.transform(y_raw.reshape(-1,1)).ravel()
        Xarr  = np.hstack([np.ones((n,1)), X_std])
        Yarr  = y_std
    else:
        Xarr = np.hstack([np.ones((n,1)), X_feat])
        Yarr = y_raw
        sx = sy = None

    p = Xarr.shape[1]  # = 1 + len(feature_cols)

    if micro_clusters is None:
        micro_clusters = 2 * n_regions
    clabel, _ = kmodels(Xarr, Yarr, micro_clusters, w,
                        init_stoc_step=False, verbose=False)
    slabel = split_components(w, clabel)
    rlabel, rcoeff, _ = greedy_merge(
        Xarr, Yarr, n_regions, w, slabel,
        min_size=min_region, verbose=False
    )
    
    bic, r2,_,_ = get_bic_r2(Xarr, Yarr, rlabel, rcoeff)

    beta_mat_std = np.vstack([rcoeff[int(lb)] for lb in rlabel])  # (n, p)
    b_std = beta_mat_std[:, 0]
    A_std = beta_mat_std[:, 1:]  # (n, m)

    df_out = df.copy()
    df_out["b_result_std"] = b_std
    for j, name in enumerate(feature_cols):
        df_out[f"{name}_coef_std"] = A_std[:, j]

    df_out["b_result"] = df_out["b_result_std"]
    for j, name in enumerate(feature_cols):
        df_out[f"{name}_coef"] = df_out[f"{name}_coef_std"]

    if save_region_label:
        df_out["region_label"] = rlabel

    if out_path:
        # stem = os.path.splitext(os.path.basename(data_path))[0]
        # out_path = os.path.join(os.path.dirname(data_path), f"{stem}_KM_result.txt")
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        df_out.to_csv(out_path, sep="\t", index=False)
        print(f"[OK] {out_path}")
    print(f"BIC: {bic},  R2: {r2}")
    return df_out, bic, r2

In [10]:
vote_path = r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Data\vote_std.txt"
df_vote = pd.read_csv(vote_path, sep="\t")

In [232]:
df_km,bic,r2 = run_kmodels_empirical(
    df = df_vote,
    y_col     = "new_pct_dem",
    w = w_q,
    feature_cols = [
        "pct_black","pct_hisp","pct_bach",
        "median_income","ln_pop_den","pct_3rd_party","pct_fb"
    ],
    coord_cols    = ("proj_X","proj_Y"),
    n_regions     = 14,
    min_region    = 20,
    micro_clusters= None,
    k_neighbors   = 4,
    standardize   = False,
    save_region_label = True,
    out_path = r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Result\vote_KM_result.txt"
)

[OK] D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Result\vote_KM_result.txt
BIC: -5680.265801014073,  R2: 0.8793686413909702


## SK

In [264]:
out_dir   = r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Result"
os.makedirs(out_dir, exist_ok=True)

#df = pd.read_csv(vote_path, sep="\t")
df = df_vote
y_col       = "new_pct_dem"
feature_cols = [
    "pct_black","pct_hisp","pct_bach",
    "median_income","ln_pop_den","pct_3rd_party","pct_fb"
]
coord_cols  = ("proj_X","proj_Y")   
n_regions   = 20
quorum      = 20
knn_k       = 8

coords = df.loc[:, coord_cols].to_numpy()
w = weights.KNN.from_array(coords, k=knn_k)
w = w.symmetrize()

X = df_vote[feature_cols].to_numpy()
y = df_vote[y_col].to_numpy()

skater = Skater_reg()
res = skater.fit(
    n_regions,
    w_q,
    X,                                 
    {"reg": PYSAL_OLS, "y": y, "x": X},
    quorum=quorum
)
labels = res._trace[-1][0].astype(int)

region_coefs = {}
for r in np.unique(labels):
    idx = (labels == r)
    Xr, yr = X[idx, :], y[idx]
    lr = LinearRegression(fit_intercept=True).fit(Xr, yr)
    region_coefs[r] = (lr.intercept_, lr.coef_)  # (b, array of betas)

b_vec  = np.array([region_coefs[int(lb)][0] for lb in labels])
B_mat  = np.vstack([region_coefs[int(lb)][1] for lb in labels])   # (N, p)

df_out = df_vote.copy()
df_out["region_label"] = labels
df_out["b_result"]     = b_vec
for j, f in enumerate(feature_cols):
    df_out[f"{f}_coef"] = B_mat[:, j]

out_path = os.path.join(out_dir, f"vote_SKREG_result.txt")
df_out.to_csv(out_path, sep="\t", index=False)
print("[OK]", out_path)

[OK] D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Result\vote_SKREG_result.txt


# SCR

# SCC

## MGWR

In [ ]:
in_path  = r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Data\vote_std_rook.txt"
out_path = r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Result\vote_MGWR_result_2.txt"

y_col       = "new_pct_dem"
coord_cols  = ("proj_X", "proj_Y")
feature_cols = [
    "pct_black", "pct_hisp", "pct_bach",
    "median_income", "ln_pop_den",
    "pct_3rd_party", "pct_fb"
]

df = pd.read_csv(in_path, sep="\t")
coords = df.loc[:, coord_cols].to_numpy()
y = df.loc[:, y_col].to_numpy().reshape(-1, 1)
X = df.loc[:, feature_cols].to_numpy()

n = len(df)

selector = Sel_BW(
    coords, y, X,
    multi=True,
    fixed=False,
    kernel="bisquare",
    spherical=False,
    constant=True,  
    #n_jobs=1
)
selector.search()

model = MGWR(
    coords, y, X, selector,
    fixed=False, kernel="bisquare", spherical=False,
    constant=True, #n_jobs=1
)
res = model.fit()

params = res.params
df_out = df.copy()
df_out["intercept"] = params[:, 0]
for j, f in enumerate(feature_cols, start=1):
    df_out[f"{f}_coef"] = params[:, j]

os.makedirs(os.path.dirname(out_path), exist_ok=True)
df_out.to_csv(out_path, sep="\t", index=False)
print(f"[OK] {out_path}")

## Sig_Test

In [ ]:
df = pd.read_csv(r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Result\vote_KM_result.txt",sep="\t")

y_col = "new_pct_dem"
x_cols = ["pct_black","pct_hisp","pct_bach",
          "median_income","ln_pop_den","pct_3rd_party","pct_fb"]
g_col = "region_label"
alpha = 0.05/14

coef_cols = ["b_result"] + [f"{x}_coef" for x in x_cols]

for g, sub in df.groupby(g_col):
    idx = sub.index
    sub = sub.dropna(subset=[y_col] + x_cols)

    if len(sub) <= len(x_cols) + 1:
        df.loc[idx, coef_cols] = np.nan
        continue

    X = sm.add_constant(sub[x_cols].to_numpy(), has_constant="add")
    y = sub[y_col].to_numpy()
    res = sm.OLS(y, X).fit()

    coefs = dict(zip(coef_cols, res.params))
    pvals = dict(zip(coef_cols, res.pvalues))

    for c in coef_cols:
        if pvals[c] >= alpha:
            coefs[c] = np.nan
        df.loc[idx, c] = coefs[c]

out_path = r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\Result\vote_KM_result_sig.txt"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
df.to_csv(out_path, sep="\t", index=False, float_format="%.6f")

# EM BIC

In [42]:
out_path = r"...\Empirical_test\BIC\km_bic_r2_2.txt"
Yarr = df_vote["new_pct_dem"].to_numpy(dtype=float).ravel()
Xarr = df_vote[["pct_black","pct_hisp","pct_bach","median_income","ln_pop_den","pct_3rd_party","pct_fb"]].to_numpy(dtype=float)
coords = df_vote[["proj_X","proj_Y"]]
# w = weights.KNN(coords, k=8)
# w = w.symmetrize()
w = w_q
n_min = 2
n_max = 20

## KM

In [76]:
Xarr = np.hstack([np.ones((len(Yarr), 1)), Xarr])
rows = []
for n_regions in range(n_min, n_max+1):  # 2..20
    K_micro = 2 * n_regions
    clabel, _ = kmodels(Xarr, Yarr, K_micro, w, init_stoc_step=False, verbose=False)
    slabel     = split_components(w, clabel)
    rlabel, rcoeff, _ = greedy_merge(
        Xarr, Yarr, n_regions, w, slabel, min_size=20, verbose=False)

    bic, r2, _, _ = get_bic_r2(Xarr, Yarr, rlabel, rcoeff)

    rows.append({"n_regions": n_regions, "BIC": bic, "R2": r2})
    print(f"n_regions={n_regions:2d} (K_micro={K_micro:2d})  BIC={bic:.6f}  R2={r2:.6f}")

df_out = pd.DataFrame(rows).sort_values("n_regions")
os.makedirs(os.path.dirname(out_path), exist_ok=True)
df_out.to_csv(out_path, sep="\t", index=False)

n_regions= 2 (K_micro= 4)  BIC=-3704.587803  R2=0.710834
n_regions= 3 (K_micro= 6)  BIC=-4538.979783  R2=0.784605
n_regions= 4 (K_micro= 8)  BIC=-4856.713269  R2=0.810519
n_regions= 5 (K_micro=10)  BIC=-4996.411475  R2=0.823482
n_regions= 6 (K_micro=12)  BIC=-5254.605333  R2=0.841714
n_regions= 7 (K_micro=14)  BIC=-5327.737506  R2=0.849348
n_regions= 8 (K_micro=16)  BIC=-5450.285102  R2=0.858878
n_regions= 9 (K_micro=18)  BIC=-5401.390258  R2=0.860302
n_regions=10 (K_micro=20)  BIC=-5529.163567  R2=0.869359
n_regions=11 (K_micro=22)  BIC=-5607.189187  R2=0.875856
n_regions=12 (K_micro=24)  BIC=-5678.556657  R2=0.881776
n_regions=13 (K_micro=26)  BIC=-5550.745955  R2=0.879958
n_regions=14 (K_micro=28)  BIC=-5651.112494  R2=0.886746
n_regions=15 (K_micro=30)  BIC=-5507.829956  R2=0.884430
n_regions=16 (K_micro=32)  BIC=-5545.679188  R2=0.888747
n_regions=17 (K_micro=34)  BIC=-5545.143489  R2=0.891572
n_regions=18 (K_micro=36)  BIC=-5453.166755  R2=0.891167
n_regions=19 (K_micro=38)  BIC=

## SK

In [77]:
skater = Skater_reg()
Yarr = df_vote["new_pct_dem"]
result = skater.fit(n_max, w_q, Xarr, {"reg": PYSAL_OLS, "y": Yarr, "x": Xarr},quorum = 20)

In [81]:
keep = {}
for rec in result._trace:
    labels = np.asarray(rec[0])
    K = np.unique(labels).size
    if 2 <= K <= n_max:
        keep[K] = labels

rows = []
for n_region in sorted(keep):
    bic, r2,_ = get_bic_r2_sk(keep[n_region], Xarr, Yarr)
    rows.append((n_region, bic, r2))
    print(f"n_regions={n_region:2d} BIC={bic:.6f}  R2={r2:.6f}")

df_out = pd.DataFrame(rows, columns=["n_regions", "BIC", "R2"]).sort_values("n_regions")
df_out.to_csv(r"D:\wanghanbin\Linear Regression Tree\code\comparision_paper\Empirical_test\BIC\skreg_bic_r2_2.txt" , sep="\t", index=False)

n_regions= 2 BIC=-3872.133946  R2=0.727434
n_regions= 3 BIC=-4031.308427  R2=0.748321
n_regions= 4 BIC=-4199.540027  R2=0.768284
n_regions= 5 BIC=-4279.837831  R2=0.780537
n_regions= 6 BIC=-4367.605563  R2=0.792642
n_regions= 7 BIC=-4445.956575  R2=0.803484
n_regions= 8 BIC=-4532.093230  R2=0.814225
n_regions= 9 BIC=-4601.878289  R2=0.823453
n_regions=10 BIC=-4624.566949  R2=0.829658
n_regions=11 BIC=-4640.197712  R2=0.835272
n_regions=12 BIC=-4652.035163  R2=0.840506
n_regions=13 BIC=-4665.449540  R2=0.845652
n_regions=14 BIC=-4675.156027  R2=0.850453
n_regions=15 BIC=-4678.078415  R2=0.854789
n_regions=16 BIC=-4724.848526  R2=0.860975
n_regions=17 BIC=-4759.894885  R2=0.866394
n_regions=18 BIC=-4767.804405  R2=0.870476
n_regions=19 BIC=-4769.568014  R2=0.874184
n_regions=20 BIC=-4755.531290  R2=0.877162


## MGWR